In [1]:
import sys, os, io
import pyodbc, pandas
import datetime

In [2]:
def writeInChunks(path, sql, cnxn, chunksize = 100000):
    """
    Writes an sql in chunk to avoid memory problems
    """ 
    with io.open(path, "w", encoding="utf-8") as fh:
        chunks = pandas.read_sql_query(sql, cnxn, chunksize=chunksize)
        next(chunks).to_csv(fh, index=False)  # write the first chunk with the column names,
                                              # but ignore the index (which will be screwed up anyway due to the chunking)
       
        for chunk in chunks:
            print('Chunk written')
            chunk.to_csv(fh, index=False, header=False) # skip the column names from now on


In [3]:
cnxn = pyodbc.connect(driver='{SQL Server}',
                      server= '192.168.27.210\CMRC',
                      database='SIOS1',
                      uid='CI',
                      pwd='123')
cur = cnxn.cursor()

In [4]:
sql = "SELECT * FROM Casos WHERE Paciente IN ( SELECT Id FROM Pacientes WHERE Identificacion LIKE 'VEN%')"
chunks = pandas.read_sql_query(sql, cnxn, chunksize=1000)


In [5]:
sql = "SELECT Id From Patients "


In [6]:
sqlVenezolanas = "SELECT Id FROM Pacientes WHERE Identificacion LIKE 'VEN%'"
def downloadPatientsAndRegistersFromPatients(cnxn, sqlPatients, pathWriteFolder):
    """
    Downloads  all the cases, registers and operations between two dates, plus/minus a certain slack to ensure all registers are fully recovered
    """
    try:
        os.mkdir(pathWriteFolder)
    except:
        pass
    sqlRegistros ="""SELECT * FROM RegistrosHistoria where NumeroHistoria in (%s)  """ % (sqlPatients)
    sqlCasos ="""SELECT * FROM Casos where Paciente in (%s) """ % (sqlPatients)
    sqlProcedimientos ="""SELECT * FROM ProgramacionQx.DescripcionQuirurgica where IdPaciente  in (%s)""" % (sqlPatients)
    sqlPacientes = """SELECT * FROM PACIENTES WHERE Id in (%s)""" %  sqlPatients
    
    sqlCasosID ="""SELECT Caso FROM Casos where Paciente in (%s) """ % (sqlPatients)
    sqlDatosEnfermeria = """SELECT * FROM Enfermeria.SignosVitalesControl WHERE IdAdmision IN (%s)""" % (sqlCasosID)

    
    writeInChunks(os.path.join(pathWriteFolder,'registros.csv'), sqlRegistros, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'casos.csv'), sqlCasos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'procedimientos.csv'), sqlProcedimientos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'pacientes.csv'), sqlPacientes, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'enfermeriaMedidas.csv'), sqlDatosEnfermeria, cnxn)


In [14]:
def downloadPatientsAndRegistersFromDate(cnxn, startDate, endDate, pathWriteFolder,  slackDays = 15):
    """
    Downloads  all the cases, registers and operations between two dates, plus/minus a certain slack to ensure all registers are fully recovered
    """
    try:
        os.mkdir(pathWriteFolder)
    except:
        pass
    slackDaysDT = datetime.timedelta(days = slackDays)
    startDateDT = datetime.datetime.strptime(startDate, '%d/%m/%Y') + slackDaysDT
    endDateDT = datetime.datetime.strptime(endDate, '%d/%m/%Y') + slackDaysDT
    
    startDateStr =  startDateDT.strftime("%Y-%d-%m")
    endDateStr =endDateDT.strftime("%Y-%d-%m")
    sqlRegistros ="""SELECT * FROM RegistrosHistoria where FechaAsignacionRegistro < '%s' AND FechaAsignacionRegistro > '%s' """ % (endDateStr, startDateStr)
    sqlCasos ="""SELECT * FROM Casos where FechaHora < '%s' AND FechaHora > '%s' """ % (endDateStr, startDateStr)
    sqlProcedimientos ="""SELECT * FROM ProgramacionQx.DescripcionQuirurgica where FechaDescripcion < '%s' AND FechaDescripcion > '%s' """ % (endDateStr, startDateStr)
    sqlProcedimientosIDs = """SELECT DISTINCT IdPaciente FROM ProgramacionQx.DescripcionQuirurgica where FechaDescripcion < '%s' AND FechaDescripcion > '%s' """ % (endDateStr, startDateStr)
    sqlPacientes = """SELECT * FROM PACIENTES WHERE Id in (%s)""" %  sqlProcedimientosIDs
    
    sqlCasosID ="""SELECT Caso FROM Casos where FechaHora < '%s' AND FechaHora > '%s' """ % (endDateStr, startDateStr)
    sqlDatosEnfermeria = """SELECT * FROM Enfermeria.SignosVitalesControl WHERE IdAdmision IN (%s)""" % (sqlCasosID)

    
    writeInChunks(os.path.join(pathWriteFolder,'registros.csv'), sqlRegistros, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'casos.csv'), sqlCasos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'procedimientos.csv'), sqlProcedimientos, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'pacientes.csv'), sqlPacientes, cnxn)
    writeInChunks(os.path.join(pathWriteFolder,'enfermeriaMedidas.csv'), sqlDatosEnfermeria, cnxn)


In [15]:
downloadPatientsAndRegistersFromDate(cnxn, '01/01/2019', '01/01/2020', 'casos2019')

Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written
Chunk written


In [16]:
cnxn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+ \
                      'DBQ=C:\\sip\\SIP_Respaldo\\sip20200302.mdb;')

In [20]:
cnxn = pyodbc.connect('Driver={Microsoft Access Driver (*.mdb, *.accdb)};'+ \
                      'DBQ=C:\\sip\\SIP_Respaldo\\sip20200302.mdb;')
cursor = cnxn.cursor()
for row in cursor.tables():
    print (row.table_name)


MSysAccessObjects
MSysAccessXML
MSysACEs
MSysIMEXColumns
MSysIMEXSpecs
MSysNavPaneGroupCategories
MSysNavPaneGroups
MSysNavPaneGroupToObjects
MSysNavPaneObjectIDs
MSysObjects
MSysQueries
MSysRelationships
________Version 414
aux_Envio
aux_Historial
aux_Informacion
aux_Instituciones_Local
aux_movimientos
aux_Tablas_x_Sistemas
aux_Usuarios
aux_Version
nivel_00
nivel_01
nivel_02
nivel_03
nivel_04
nivel_05
nivel_06
nivel_11
nivel_12
nivel_13
nivel_16
nivel_21
nivel_22
nivel_31
nivel_32
nivel_33
nivel_34
nivel_41
nivel_51
nivel_52
nivel_55
nivel_56
nivel_57
nivel_61
nivel_71
nivel_72
nivel_73
nivel_74
nivel_75
nivel_76
nivel_77
nivel_81
nivel_82
nivel_83
nivel_84
nivel_85
nivel_86
nivel_91
nivel_92
nivel_93
nivel_94
nivel_95
nivel_96
nivel_A1
nivel_A2
nivel_A3
nivel_N1
nivel_N2
nivel_N3
nivel_N4
nivel_N5
nivel_N6
nivel_N7
nivel_N8
nivel_S1
nivel_S2
nivel_S3
nivel_S4
nivel_S5
nivel_S6
nivel_T1
nivel_T2
nivel_T3
nivel_T4
nivel_T5
nivel_T6
nivel_z0
Consulta_01
Consulta_02
Consulta_03
Consulta_

In [24]:
df = pandas.read_sql_query("select top 10 * from Nivel_N1", cnxn)
for c in df.columns:
    print(c)

ID01
ID02
ID03
VAR_0001
VAR_0002
VAR_0003
VAR_0004
VAR_0005
VAR_0006
VAR_0009
VAR_0010
VAR_0011
VAR_0012
VAR_0013
VAR_0014
VAR_0015
VAR_0016
VAR_0017
VAR_0018
VAR_0019
VAR_0020
VAR_0022
VAR_0024
VAR_0026
VAR_0028
VAR_0030
VAR_0021
VAR_0023
VAR_0025
VAR_0027
VAR_0029
VAR_0031
VAR_0032
VAR_0033
VAR_0034
VAR_0035
VAR_0036
VAR_0037
VAR_0040
VAR_0038
VAR_0039
VAR_0041
VAR_0045
VAR_0046
VAR_0042
VAR_0047
VAR_0043
VAR_0044
VAR_0048
VAR_0049
VAR_0050
VAR_0051
VAR_0052
VAR_0053
VAR_0054
VAR_0055
VAR_0056
VAR_0057
VAR_0058
VAR_0059
VAR_0060
VAR_0080
VAR_0081
VAR_0082
VAR_0083
VAR_0084
VAR_0085
VAR_0086
VAR_0087
VAR_0088
VAR_0089
VAR_0090
VAR_0095
VAR_0096
VAR_0097
VAR_0098
VAR_0099
VAR_0100
VAR_0101
VAR_0102
VAR_0103
VAR_0104
VAR_0105
VAR_0106
VAR_0108
VAR_0107
VAR_0109
VAR_0110
VAR_0111
VAR_0093
VAR_0112
VAR_0115
VAR_0114
VAR_0182
VAR_0183
VAR_0184
VAR_0185
VAR_0186
VAR_0187
VAR_0188
VAR_0189
VAR_0190
VAR_0191
VAR_0192
VAR_0193
VAR_0194
VAR_0195
VAR_0196
VAR_0197
VAR_0198
VAR_0199
VAR_0200
VAR_

In [5]:
diagnosis =  pandas.read_sql_query('SELECT * FROM Procedimientos ', cnxn)
diagnosis.to_csv('procedimientosID.csv', index=False)



In [4]:
diagnosis =  pandas.read_sql_query('SELECT * FROM Diagnosticos ', cnxn)
diagnosis.to_csv('diagnosis.csv', index=False) # skip the column names from now on


In [ ]:
diagnosis =  pandas.read_sql_query('SELECT * FROM Diagnosticos ', cnxn)
diagnosis.to_csv('diagnosis.csv', index=False) # skip the column names from now on


In [40]:
diagnosis.loc[diagnosis['CodDiag'] == 'Z370']

,CodDiag,Simbolo,Descrip,Sexo,LimInf,LimSup,NoAfeccion,Observacion,Clasificacion
11989,Z370,None,"NACIDO VIVO, UNICO",A,NaN,227.0,None,None,CIE10


In [37]:
import xml.dom.minidom

dom = xml.dom.minidom.parseString(registers.loc[8, 'RegistroXML']) # or xml.dom.minidom.parseString(xml_string)
pretty_xml_as_string = dom.toprettyxml()
print(pretty_xml_as_string)

<?xml version="1.0" ?>
<C Asunto="Evoluci&amp;#243;n" Caso="AD236491" CentroA="01" CodigoDiagnosticoRelacionado1="" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O623" IdPaciente="170478" Modulo="01" Padre="860231" PlanAdm="480110" Prestador="73102778" Raiz="94" Registro="106">
	<row NombreCampo="DescripcionNota" NombreTabla="CamposMemo" ValorCampo="SE TRASLADA A SALA DE PARTO #1 PARA REALIZACION DE AMNIOTOMIA.&amp;lt;BR&amp;gt;EN POSICION DE LITOTOMIA PREVIA ASEPSIA, ANTISEPSIA CON PRESENCIA DE FRECUENCIA CARDIACA FETAL PREAMNIOTOMIA DE 142 LPM SE REALIZA AMNIOTOMIA OBTENIENDO LIQUIDO CLARO, EUTERMICO, NO FETIDO, EN ABUNDANTE CANTIDAD, SE VERICA FRECUENCIA CARDIACA FETAL POSTERIOR A LA AMNIOTOMIA EN 144LPM. CONTINUAR CON VIGILANCIA DE TRABAJO DE PARTO."/>
	<row NombreCampo="FechaNota" NombreTabla="CamposFecha" ValorCampo="06/07/2017 13:37"/>
	<row NombreCampo="FechaEgreso" NombreTabla="CamposFecha" ValorCampo=""/>
	<row NombreCampo="Autorizacion" Nombr

In [26]:
for i,p in enumerate(registers['CodigoDiagnosticoRelacionado1']):
    print(i,  p)

0 None
1 Z014
2 Z014
3 None
4 
5 
6 
7 None
8 
9 None
10 Z370
11 None
12 None
13 
14 R104
15 None
16 Z488
17 D509


In [44]:
import pandas
DF = pandas.read_sql_query("SELECT TOP 5 * FROM RegistrosHistoria", cnxn)


In [45]:
DF

,Id,NumeroHistoria,Caso,CodigoRegistro,Asunto,FechaAsignacionRegistro,Raiz,Padre,Usuario,Prestador,...,AltaMedica,IdRegistroPreanestesico,IdRegistroIntraOperatorio,IdRegistroPostOperatorio,Modulo,ProgramaPyP,FechaInicioAtencion,UnidadFuncional,ConfirmacionGuardado,Sincronizado
0,2625,68909,AD0004,07,Ingreso de Urgencias,2013-09-20 09:14:12.747,NULL,NULL,sa,73083700,...,False,None,None,None,03,None,None,None,False,None
1,2626,68909,TRI001,25,,2013-09-20 09:14:13.147,None,None,sa,73083700,...,False,None,None,None,None,None,None,None,False,None
2,2627,68909,AD0004,02,MOTIVACION PARA CESAREA,2013-09-20 10:04:44.483,NULL,NULL,sa,73083700,...,False,None,None,None,03,None,None,None,False,None
3,2628,68909,AD0004,14,EVOLUCION,2013-09-20 11:06:56.620,07,2625,sa,73083700,...,False,None,None,None,03,None,None,None,False,None
4,2629,68930,AD0012,02,SDEW,2013-09-20 16:52:21.880,NULL,NULL,sa,73080532,...,False,None,None,None,01,None,None,None,False,None


In [ ]:
columns = [column[0] for column in cur.description]

In [ ]:
columns

In [ ]:
conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=C:\sip\SIP_respaldo\sip20200207.mdb;')


In [ ]:
os.path.exists(MDB)

In [ ]:
cnxn = pyodbc.connect(driver='{SQL Server}', host=SRV, database=MDB,
                      trusted_connection=True)


In [ ]:
import subprocess
pathMDBTools = 'C:\\Users\\Public\\Gabriel\\mdbtools-win-master\\mdbtools-win-master'
DATABASE = 'C:\\sip\\SIP_respaldo\\sip20200207.mdb'
#!/usr/bin/env python
#
# AccessDump.py
# A simple script to dump the contents of a Microsoft Access Database.
# It depends upon the mdbtools suite:
#   http://sourceforge.net/projects/mdbtools/

import sys, subprocess # the subprocess module is new in python v 2.4

# Get the list of table names with "mdb-tables"
table_names = subprocess.Popen([os.path.join(pathMDBTools,"mdb-tables.exe"), "-1", DATABASE], 
                               stdout=subprocess.PIPE).communicate()[0]
tables = table_names.split(b'\n')

# Dump each table as a CSV file using "mdb-export",
# converting " " in table names to "_" for the CSV filenames.
for table in tables[1:]:
    if table != '':
        table = table.decode("utf-8").replace('\r', '')
        filename = table.replace(" ","_") + ".csv"
        file = open(filename.replace('\r', ''), 'w')
        print("Dumping " + table)
        contents = subprocess.Popen([os.path.join(pathMDBTools, "mdb-export"), DATABASE, table],
                                    stdout=subprocess.PIPE).communicate()[0]
        file.write(contents.decode("utf-8"))
        file.close()

In [ ]:
print(os.path.join(pathMDBTools,"mdb-tables.exe"), "-1", DATABASE)

In [ ]:
contents